<a href="https://colab.research.google.com/github/Akilesh1989/music_generation_with_deep_learning/blob/master/Generate_music_using_Deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**This notebook was inspired (and almost all the code comes from it) by [towardsdatascience](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5)**

**Files used in this notebook can be found at [github](https://github.com/unmonoqueteclea/DeepLearning-Notebooks/tree/master/LSTM-Music-Generation)**

In [1]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install music21;

In [ ]:
# Only upload midi files
from google.colab import files
files.upload()


Saving Movie_Themes_-_20th_Century_Fox.midi to Movie_Themes_-_20th_Century_Fox.midi


unzip:  cannot find or open midi_files.zip, midi_files.zip.zip or midi_files.zip.ZIP.


## Processing data

Let's process the files, and load them into **music21**

In [3]:
# Importing dependencies
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [4]:
file_name = "jingle-bells-keyboard.midi"
file = f"/content/drive/My Drive/Projects/Music_generation/Input/{file_name}"
midi = converter.parse(file) # parse the file
notes_to_parse = midi.flat.notes # get notes
for element in notes_to_parse[:10]:
  print(element, element.offset)

<music21.chord.Chord D4 G2> 0.0
<music21.note.Note B> 0.0
<music21.chord.Chord B4 D3> 0.5
<music21.chord.Chord A4 G3> 1.0
<music21.chord.Chord G4 D3> 1.5
<music21.note.Note D> 2.0
<music21.note.Note B> 2.0
<music21.note.Note G> 2.0
<music21.note.Note D> 2.5
<music21.note.Note G> 3.0


In [5]:
notes = []
midi = converter.parse(file)
print('\r', 'Parsing file '," ",file, end='')
notes_to_parse = None
try: # file has instrument parts
  s2 = instrument.partitionByInstrument(midi)
  print("\n")
  print(s2.parts)
  notes_to_parse = s2.parts[0].recurse() 
except: # file has notes in a flat structure
  notes_to_parse = midi.flat.notes
print("\n")
for element in notes_to_parse:
  
  print(type(element))
  if isinstance(element, note.Note):
    notes.append(str(element.pitch))
  elif isinstance(element, chord.Chord):
    notes.append('.'.join(str(n) for n in element.normalOrder))


with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

 Parsing file    /content/drive/My Drive/Projects/Music_generation/Input/jingle-bells-keyboard.midi

<music21.stream.iterator.StreamIterator for Score:0x7f5b9009e710 @:0>


<class 'music21.instrument.Piano'>
<class 'music21.tempo.MetronomeMark'>
<class 'music21.key.Key'>
<class 'music21.meter.TimeSignature'>
<class 'music21.chord.Chord'>
<class 'music21.note.Note'>
<class 'music21.chord.Chord'>
<class 'music21.chord.Chord'>
<class 'music21.chord.Chord'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.chord.Chord'>
<class 'music21.note.Note'>
<class 'music21.chord.Chord'>
<class 'music21.chord.Chord'>
<class 'music21.chord.Chord'>
<class 'music21.chord.Chord'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.note.Note'>
<class 'music21.chord.Chord'

In [6]:
def get_notes_chords_rests(midi, instrument_type, path):
    try:
        parts = instrument.partitionByInstrument(midi)
        print(len(parts))
        print(parts.parts['Piano'].id)
        for a in parts.parts:
          print(a)
        note_list = []
        for music_instrument in range(len(parts)):
            if parts.parts[music_instrument].id in instrument_type:
                print("Match")
                for element_by_offset in stream.iterator.OffsetIterator(parts[music_instrument]):
                    print(element_by_offset)
                    for entry in element_by_offset:
                        if isinstance(entry, note.Note):
                            note_list.append(str(entry.pitch))
                        elif isinstance(entry, chord.Chord):
                            note_list.append('.'.join(str(n) for n in entry.normalOrder))
                        elif isinstance(entry, note.Rest):
                            note_list.append('Rest')
        return note_list
    except Exception as e:
        print("failed on ", path)
        pass

keyboard_instrument = ["Piano"]
midi = converter.parse(file)
notes = get_notes_chords_rests(midi, keyboard_instrument, file)
notes

1
Piano
<music21.stream.Part Piano>
Match
[<music21.instrument.Piano Piano>, <music21.tempo.MetronomeMark Quarter=105.0>, <music21.key.Key of G major>, <music21.meter.TimeSignature 2/4>, <music21.chord.Chord D4 G2>, <music21.note.Note B>]
[<music21.chord.Chord B4 D3>]
[<music21.chord.Chord A4 G3>]
[<music21.chord.Chord G4 D3>]
[<music21.note.Note D>, <music21.note.Note B>, <music21.note.Note G>]
[<music21.note.Note D>]
[<music21.note.Note G>]
[<music21.note.Note D>, <music21.note.Note D>]
[<music21.note.Note D>]
[<music21.chord.Chord D4 G2>, <music21.note.Note B>]
[<music21.chord.Chord B4 D3>]
[<music21.chord.Chord A4 G3>]
[<music21.chord.Chord G4 D3>]
[<music21.chord.Chord E4 C4>, <music21.note.Note C>]
[<music21.note.Note E>]
[<music21.note.Note G>]
[<music21.note.Note E>]
[<music21.chord.Chord E4 A2>, <music21.note.Note C>]
[<music21.chord.Chord C5 E3>]
[<music21.chord.Chord B4 A3>]
[<music21.chord.Chord A4 E3>]
[<music21.chord.Chord F#4 D4>, <music21.note.Note D>]
[<music21.note.No

['2.7',
 'B3',
 '11.2',
 '7.9',
 '2.7',
 'D4',
 'B3',
 'G2',
 'D3',
 'G3',
 'D4',
 'D3',
 'D4',
 '2.7',
 'B3',
 '11.2',
 '7.9',
 '2.7',
 '0.4',
 'C3',
 'E3',
 'G3',
 'E3',
 '4.9',
 'C4',
 '0.4',
 '9.11',
 '4.9',
 '2.6',
 'D3',
 'F#3',
 'A3',
 'F#3',
 '2',
 'F#4',
 '2.6',
 '9.0',
 'F#4',
 '9.2',
 '7.11',
 'G2',
 'D3',
 'G3',
 'D3',
 '2.7',
 'B3',
 '11.2',
 '7.9',
 '2.7',
 'D4',
 'B3',
 'G2',
 'D3',
 'G3',
 'D4',
 'D3',
 'D4',
 '2.7',
 'B3',
 '11.2',
 '7.9',
 '2.7',
 '0.4',
 'C3',
 'E3',
 'G3',
 'E3',
 '4.9',
 'C4',
 '0.4',
 '9.11',
 '4.9',
 '11.2',
 'G4',
 '2',
 '2.7',
 'G4',
 '2',
 '4.9',
 'F#4',
 '2',
 '0.6',
 'F#4',
 '9.2',
 '2.7',
 'G2',
 'D3',
 '2.7',
 '7.11',
 'G2',
 '7.11',
 '7.11',
 'D3',
 '7.11',
 'G3',
 '7.11',
 '7.11',
 'D3',
 '7.11',
 'G4',
 '2',
 'G4',
 '0.4',
 'A4',
 '11.2',
 'G2',
 'D3',
 'G3',
 '0.4',
 '0.4',
 'C5',
 'E4',
 'G3',
 'E3',
 'C5',
 '11.0',
 'D4',
 '11.2',
 '7.11',
 'D4',
 'B4',
 'D3',
 'B4',
 '9.11',
 'C#4',
 '4.9',
 '9',
 'C#4',
 '11.4',
 '9.2',
 'D3',
 'F#

In [7]:
instruments = []
midi = converter.parse(file)  
parts = instrument.partitionByInstrument(midi) 
notes_for_instruments = []
for i in range(len(parts.parts)): 
    notes_to_parse = parts.parts[i].recurse()
    instr = parts.parts[i].getInstrument()
    instruments.append(instr.instrumentName)
    notes, quarters = [],[] 
    for element in notes_to_parse:   
        if isinstance(element, note.Note):  
        # if element is a note, extract pitch   
            notes.append(str(element.pitch))
        elif(isinstance(element, chord.Chord)):  
            # if element is a chord, append the normal form of the   
            # chord (a list of integers) to the list of notes.   
            notes.append('.'.join(str(n) for n in element.normalOrder))  
        elif isinstance(element, note.Rest):
            notes.append('Rest')

    notes_for_instruments.append(notes)

We obtain the number of different notes in our dataset, because this will be the **number of possible output classes**  of our model.

In [8]:
# Count different possible outputs
n_vocab = (len(set(notes)))
n_vocab

32

In [9]:
len(notes)

360

Now, there is some **data processing** that we have to do:

- We will map each pitch or chord to an integer
- We will create pairs of input sequences and its corresponding output note

We can try different **sequence_length** to obtain different results. In this first version, we will use a sequence_length of 100.

The network will made its prediction of the next note (or chord), based on the previous *sequence_length* notes (or chords). 

![Sequence learning](https://unmonoqueteclea.github.io/assets/images/inputoutputsequences.png)

In [10]:
sequence_length = 10
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length] # Size sequence_length
  sequence_out = notes[i + sequence_length]  # Size 1
  # Map pitches of sequence_in to integers
  network_input.append([note_to_int[char] for char in sequence_in])
  # Map integer of sequence_out to an integer
  network_output.append(note_to_int[sequence_out])
  n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [11]:
network_output

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

Let's see the new network_input size

In [13]:
network_input.shape

(350, 10, 1)

## Creating model

Let's create the network. We will create a network with 9 layers (3 of them **LSTM layers**).

For regularization, we will also add 2 **Dropout** layers

In [14]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [15]:
model = create_network(network_input,n_vocab)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 512)           1052672   
_________________________________________________________________
dropout (Dropout)            (None, 10, 512)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 512)           2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In case we want to use previously trained weights, to continue the training in the point we left it, we should load them into the model. 

This is very useful in Google Colaboratory, that usually kills the virtual machine that is executing the Jupyter notework after a certime amount of time. If this happens to you, you should have to look for the last weights file in your configured Drive account and use it to train the network.

In [ ]:
# In case we want to use previously trained weights
weights = ""
if(len(weights)>0): model.load_weights(weights)

We will use **ModelCheckpoint**.

ModelCheckpoint will save our weights in a file after each epoch.

This way, we can start execution where we left if the training stops.

You can train as many epochs as you want. I have checked that, with **75 epochs**, the network is able to compose new interesting music. You can do this 75 epochs using Google Colab GPU in about 13 hours. As you can see, you don't need an extremly low loss, 0.7 or 0.8 is fine.





In [ ]:
filepath = "/content/drive/My Drive/Projects/Music_generation/jb_3/{epoch:02d}-{loss:.4f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=0,
                             save_best_only=True,mode='min')

callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=400, batch_size=64, 
          callbacks=callbacks_list)

Epoch 1/400
6/6 [==============================] - 10s 19ms/step - loss: 3.5087
Epoch 2/400
6/6 [==============================] - 0s 19ms/step - loss: 3.4531
Epoch 3/400
6/6 [==============================] - 0s 17ms/step - loss: 3.3191
Epoch 4/400
6/6 [==============================] - 0s 18ms/step - loss: 3.2494
Epoch 5/400
6/6 [==============================] - 0s 16ms/step - loss: 3.2376
Epoch 6/400
6/6 [==============================] - 0s 15ms/step - loss: 3.2615
Epoch 7/400
6/6 [==============================] - 0s 15ms/step - loss: 3.1937
Epoch 8/400
6/6 [==============================] - 0s 13ms/step - loss: 3.2818
Epoch 9/400
6/6 [==============================] - 0s 13ms/step - loss: 3.2378
Epoch 10/400
6/6 [==============================] - 0s 13ms/step - loss: 3.0629
Epoch 11/400
6/6 [==============================] - 0s 13ms/step - loss: 3.0933
Epoch 12/400
6/6 [==============================] - 0s 13ms/step - loss: 3.0414
Epoch 13/400
6/6 [==============================

## Music generation

Let's compose music!
We have renamed our last weights file as **final-weights.h5**.

 **FILE DOWNLOAD: https://github.com/unmonoqueteclea/DeepLearning-Notebooks/tree/master/LSTM-Music-Generation**

There can be generated songs that sound awful, but try to execute the generation
process several times and you will get interesting results.

![Music Generation](https://unmonoqueteclea.github.io/assets/images/lstm.png)

In [ ]:
import os
# In case we want to use other previously trained weights
file_path = "/content/drive/My Drive/Projects/Music_generation/jb_3/"
max_file = max([int(file.split("-")[0]) for file in os.listdir(file_path)])
for file in os.listdir(file_path):
  if file.startswith(str(max_file)):
    weights_file = file

weights_file  
weights = f"{file_path}/{weights_file}"
if(len(weights)>0): model.load_weights(weights)

In [ ]:
# Generate network input again
network_input = []
output = []
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

The workflow now is:


1.   Pick a **seed sequence** randomly from your list of inputs (*pattern* variable)
2.   Pass it as input for your model to generate a new element (note or chord)
3.   Add the new element to your final song and to your *pattern* list
4.   Remove the first item from *pattern*
5.   Go to step 2




In [ ]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for i,note_index in enumerate(range(500)):
  prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
  prediction_input = prediction_input / float(n_vocab)
  prediction = model.predict(prediction_input, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result)
  prediction_output.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

The last step is creating a MIDI file from the predictions.

**music21** will help us again for this task. We should create a **Stream** and add to it the predicted notes and chords.

We are adding an offset of 0.5 between elements.

In [ ]:
pattern

[23, 27, 6, 15, 27, 9, 29, 9, 9, 23]

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'